# পর্ব ৪ঃ মডেল গড়ের মাধ্যমে ফেডারেটেড লার্নিং

**ফিরে দেখা:** টিউটোরিয়ালের পর্ব-২ এ আমরা ফেডারেটেড লার্নিং এর সাধারণ ভার্সন দিয়ে একটি মডেল ট্রেইন করেছিলাম। এক্ষেত্রে প্রতিটি তথ্যের অধিকারীকে তাদের গ্রেডিয়েন্টের জন্য মডেল অধিকারীর উপর বিশ্বাস রাখা প্রয়োজনীয় ছিল।

**বিবরণ:** এই টিউটোরিয়ালে আমরা দেখবো - চূড়ান্ত মডেলকে মডেল অধিকারীর কাছে পাঠানোর আগে কিভাবে পর্ব ৩ এর উন্নীত একত্রিকরণ সরঞ্জাম সমূহ ব্যবহার করে বিশ্বস্ত "secure worker" এর মাধ্যমে ওয়েটগুলো(weights) একত্রিকরণ করা হয়।

এই পন্থায় শুধুমাত্র নিরাপদ কর্মী দেখতে পারে কার ওয়েটগুলো(weights) কার কাছ থেকে এসেছে। আমরা হয়তো বলবো পারবো মডেল কোথায় কি পরিবর্তন এসেছে কিন্তু আমরা জানতে পারবো না কোন কর্মী কোন পরিবর্তনের জন্য দায়ী। যা কি-না গোপনীয়তার একটি আস্তরণ তৈরি করে।

লেখকঃ
 - Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
 - Jason Mancuso - Twitter: [@jvmancuso](https://twitter.com/jvmancuso)

অনুবাদকঃ
 - মীর মোহাম্মদ জাবের(Mir Mohammad Jaber) - Twitter: [@jabertuhin](https://twitter.com/jabertuhin)

In [0]:
import torch
import syft as sy
import copy
hook = sy.TorchHook(torch)
from torch import nn, optim

# ধাপ ১ঃ ডাটা অধিকারী তৈরি করি

প্রথমে আমরা অল্প কিছু তথ্য দিয়ে বব(Bob) ও এলিস(Alice) নামে দু'টি তথ্য অধিকারী তৈরি করি। আমরা এর সাথে "secure_worker" নামে একটি নিরাপদ মেশিন তৈরি করি। কাজের ক্ষেত্রে এটি নিরাপদ কোন হার্ডওয়্যার(যেমন - Intel's SGX) কিংবা সাধারণ কোন বিশ্বস্ত মাধ্যম হতে পারে।

In [0]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

# ধাপ ২ঃ আমাদের মডেল তৈরি করি

এই উদাহরণে আমরা একটি সাধারণ লিনিয়ার মডেল তৈরি করবো। আমরা এটি Pytorch এর nn.Linear কন্সট্রাকটর ব্যবহার করে তৈরি করতে পারি।

In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

# ধাপ ৩ঃ মডেলের একটা অনুলিপি বব(Bob) ও এলিস(Alice) কে পাঠাই

অতঃপর মডেলের একটা অনুলিপি আমরা বব(Bob) ও এলিস(Alice) কে দিই যেন তারা নিজেদের ডাটাসেটের উপর লার্নিং পদক্ষেপগুলো চালাতে পারে।

In [0]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

# ধাপ ৪ঃ বব(Bob) ও এলিস(Alice) এর মডেল ট্রেইন করা(সমান্তরালি)

ফেডারেটেড লার্নিং এর মাধ্যমে প্রচলিতভাবে যেভাবে নিরাপদ গড় করা হয়ে থাকে - দু'টি মডেল গড় করে একত্রিত করার আগে প্রত্যেক তথ্যের অধিকারী প্রথমে তাদের মডেল কয়েকবার করে স্থানীয়ভাবে ট্রেইন করে।

In [0]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

# ধাপ ৫ঃ উভয় হালনাগাদকৃত(updated) মডেলকে নিরাপদ কর্মীর কাছে পাঠানো

এখন প্রতিটা তথ্যের অধিকারী এর কাছে আংশিক ট্রেইনড মডেল আছে, এই সময় সেগুলো নিরাপদভাবে গড় করে একত্রিত করতে হবে। বব(Bob) ও এলিস(Alice) কে তাদের মডেল নিরাপদ (বিশ্বস্ত) সার্ভারে পাঠাতে বলার মাধ্যমে আমরা তা করতে পারি।
খেয়াল করি, আমাদের API এর এই ব্যবহারের মাধ্যমে প্রতিটা মডেল সরাসরি secure_worker এর কাছে চলে যায়। আমরা কখনো তা দেখতে পারি না।

In [0]:
alices_model.move(secure_worker)

In [0]:
bobs_model.move(secure_worker)

# ধাপ ৬ঃ মডেলগুলোর গড় করি

পরিশেষে, এই ট্রেইনিং ইপোক(epoch) এর শেষ ধাপে বব ও এলিসের ট্রেইনড মডেল দু'টো গড় করে একত্রিত করবো এবং এর মাধ্যমে আমাদের গ্লোবাল মডেলের মান নির্ধারণ করবো।

In [0]:
with torch.no_grad():
    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())


# পরিষ্কার করে পুনরাবৃত্তি

আর এখন আমাদেরকে শুধু এই কাজ কয়েকবার করে করতে হবে।

In [0]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

    for wi in range(worker_iters):

        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target)**2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target)**2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data
    
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    with torch.no_grad():
        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

সবশেষে আমরা নিশ্চিত হতে চাই যে আমাদের মডেল ঠিকঠাকভাবে শিখেছে। তাই আমরা এটিকে টেস্ট ডাটাসেটের উপরে মূল্যায়ন করবো। আমাদের তৈরি করা এই মিছে সমস্যার ক্ষেত্রে আমরা আসল তথ্যই ব্যবহার করছি। কিন্তু বাস্তবে আমরা নতুন তথ্য দিয়ে বুঝতে চেষ্টা করি আমাদের মডেল অদেখা উদাহরণ কতটা জেনারেলাইজ করে।

In [0]:
preds = model(data)
loss = ((preds - target) ** 2).sum()

In [0]:
print(preds)
print(target)
print(loss.data)

আমাদের এই উদাহরণের ক্ষেত্রে, গড় মডেল সাধারণ স্থানীয় ট্রেইন্ড মডেলের থেকে বেশি আন্ডারফিট(underfit) করছে। যদিও আমরা তা করতে পেরেছি কোন কর্মীর ট্রেইনিং ডাটা অন্য আরেকজনের কাছে উন্মুক্ত না করে। আমরা কোন তথ্য মডেল অধিকারীর কাছে প্রকাশ না করেই, হালনাগাদকৃত মডেলগুলো একটি বিশ্বস্ত সংযোগকারীর মাধ্যমে একত্রিত করতে পেরেছি।

ভবিষ্যত টিউটোরিয়ালে, আমাদের লক্ষ্য থাকবে সরাসরি গ্রেডিয়েন্টে বিশ্বস্ত একত্রিকরণ করতে পারা। যেন আমরা মডেল আরো ভালো গ্রেডিয়েন্ট এস্টিমেটস দিয়ে হালনাগাদ করতে পারি এবং আরো জোরালো মডেল পাই।

# অভিনন্দন!!! - আমাদের কমিউনিটিতে যোগদান করুন

এই নোটবুক টিউটোরিয়ালটি শেষ করার জন্য অভিনন্দন! আপনি যদি এটি পছন্দ করে থাকেন এবং গোপনীয়তা সংরক্ষণ, কৃত্রিম বুদ্ধিমত্তার অধিকারের বিকেন্দ্রিকরণ এবং এর সাপ্লাই চেইনের(তথ্য) আন্দোলনে যোগ দিতে চান তাহলে আপনি নিম্নোক্ত পন্থা অনুসারে তা করতে পারেন।

### PySyft -কে গিটহাবে স্টার দিন

আমাদের কমিউনিটিকে সাহায্য করার সবচেয়ে সহজ পন্থা হলো রিপোজিটোরিতে স্টার দেয়া! এটি আমরা যে দারুন সরঞ্জাম তৈরি করছি সে ব্যাপারে সচেতনতা বৃদ্ধি করতে সাহায্য করে।

- [Star PySyft](https://github.com/OpenMined/PySyft)

### আমাদের স্ল্যাকে(Slack) যোগ দিন!

নতুন নতুন অগগ্রতির সাথে নিজেকে টিকিয়ে রাখার সেরা উপায় হলো আমাদের কমিউনিটিতে যোগ দেয়া! আর তা করার জন্য উল্লিখিত ফর্মটি পূরণ করতে হবে [http://slack.openmined.org](http://slack.openmined.org)

### কোড প্রজেক্টে যোগ দিন!

আমাদের কমিউনিটিতে অবদান রাখার সেরা উপায় হলো একজন কোড অবদানকারীতে পরিণত হওয়া। যেকোন সময় আপনি PySyft এর গিটহাব ইস্যুর পেজে যেতে পারেন এবং "Projects" দিয়ে বাছাই করবেন। এর মাধ্যমে আপনি যে সকল প্রজেক্টে যোগদান করতে পারবেন সেগুলোর উপরের দিকের টিকেটের ওভারভিউ পাবেন। আপনি যদি কোন প্রজেক্টে জয়েন করতে ইচ্ছুক না হোন, কিন্তু কিছুটা কোডিং করতে ইচ্ছুক সেক্ষেত্রে আপনি "one off" মিনি প্রজেক্টগুলো দেখতে পারেন গিটহাব ইস্যুতে "good first issue" চিহ্নিত ইস্যুগুলো।

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### দান করুন

আপনার যদি আমাদের কোডবেজে অবদান রাখারা সময় না হয়, কিন্তু এরপরও আমাদেরকে সমর্থন দিতে চান তাহলে আমাদের উন্মুক্ত সংগ্রহের সমর্থক হতে পারেন। সকল ধরনের দানের অর্থ আমাদের ওয়েব হোস্টিং এবং অন্যান্য কমিউনিটি কার্যক্রমে খরচ হয় যেমন - হ্যাকাথন, মিটাপ।

[OpenMined's Open Collective Page](https://opencollective.com/openmined)